# Simple model simulating trading between two agents - not deployed contract

In [57]:
import sys
sys.path.append("..")
import pandas as pd
import plotly.express as px
from radcad import Model, Simulation, Experiment
from radcad.engine import Engine, Backend
import random
from cadEVM.connector  import connect_fork, connect_mainnet
from ape import config, networks, accounts, project, chain, Contract
from dotenv import load_dotenv


In [22]:
# remember to have your alchemy 
load_dotenv()


True

In [23]:
!ape compile

In [24]:
connect_fork()

<hardhat chain_id=31337>

In [25]:
dev = accounts.test_accounts[0]
reciever_1 = accounts.test_accounts[1]

In [26]:
dev, reciever_1

(<TestAccount 0x1e59ce931B4CFea3fe4B875411e280e173cB7A9C>,
 <TestAccount 0xc89D42189f0450C2b2c3c61f58Ec5d628176A1E7>)

In [27]:
networks.network

<ethereum:mainnet-fork chain_id=31337>

In [28]:
contract = project.Token.deploy("Test", "TST", 18, 2000000, sender = dev)

INFO: Confirmed 0x87e75f958402d49736b6c4b8fc2f19a6975b3880030107f9a44d6595ce3f0a67 (total fees paid = 521433000000000)
SUCCESS: Contract 'Token' deployed to: 0xe586Ca95C4f80436D8E265A2689AeAD4e0c21011


In [29]:
chain.provider.get_block("latest")

Block(num_transactions=1, hash=HexBytes('0xb1c5e63fb4872da6559f0942b5ab04c2936dc0147166731c60b467f15f25ef81'), number=18541791, parent_hash=HexBytes('0x77800cbe6232fe4590a77e8255cdbc6358c7997371527621fa0e22402222d832'), size=3189, timestamp=1699621226, gas_limit=30000000, gas_used=521433, base_fee=0, difficulty=0, total_difficulty=58750003716598352816469)

In [30]:
contract.totalSupply()

2000000

In [31]:
contract.balanceOf(dev)

2000000

In [32]:
contract.transfer( accounts.test_accounts[1] , 100000, sender = dev)

INFO: Confirmed 0xe91d50e2c63ee952bac4bf9d1ef059d605f182080a61d9cc08b1b59845a0e931 (total fees paid = 51634000000000)


<Receipt 0xe91d50e2c63ee952bac4bf9d1ef059d605f182080a61d9cc08b1b59845a0e931>

In [33]:
params = {
    'a_agent': dev,
    'b_agent': reciever_1,
}
params

{'a_agent': <TestAccount 0x1e59ce931B4CFea3fe4B875411e280e173cB7A9C>,
 'b_agent': <TestAccount 0xc89D42189f0450C2b2c3c61f58Ec5d628176A1E7>}

In [34]:
initial_state =  {
      
     'a_balance': contract.balanceOf(params['a_agent']),
      'b_balance': contract.balanceOf(params['b_agent'])
}
initial_state

{'a_balance': 1900000, 'b_balance': 100000}

In [35]:
def p_trade(params, substep, state_history, prev_state, **kwargs):
    global contract
   
    if random.choice([True, False]):
        sender_address = params['a_agent']
        recipient_address = params['b_agent']
    else:
        sender_address = params['b_agent']
        recipient_address = params['a_agent']

    try:
        contract.transfer(recipient_address , 50000, sender = sender_address)
    except:
        contract.transfer(recipient_address , 50000, sender = dev)

    sent = "Sent"

    return {'sent_signal': sent}

In [36]:
def s_update_balance_a(params, substep, state_history, prev_state, policy_input, **kwargs):
    balance = contract.balanceOf(params['a_agent'])
    return 'a_balance', balance 

def s_update_balance_b(params, substep, state_history, prev_state, policy_input, **kwargs):
    balance = contract.balanceOf(params['b_agent'])
    return 'b_balance', balance 


In [37]:
state_update_blocks = [
    {
        'policies': {
            'trade': p_trade,
        },
        'variables': {
            'a_balance': s_update_balance_a,
            'b_balance': s_update_balance_b,
        }
    }

]

In [38]:
TIMESTEPS = 100
RUNS = 1
model = Model(initial_state=initial_state, state_update_blocks=state_update_blocks, params=params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)
experiment = Experiment(simulation)
# Select the Pathos backend to avoid issues with multiprocessing and Jupyter Notebooks
experiment.engine = Engine(backend=Backend.PATHOS)

result = experiment.run()

INFO: Confirmed 0xa535d52e4b61a2a2bf612b5aade7a9f9c5855b61909dd2acf3656d08b72ca4aa (total fees paid = 34522000000000)
INFO: Confirmed 0xad7f64e356837530603b1927cdb7d2b9f7ab650ed42ba2d4d369a1348573544d (total fees paid = 34522000000000)
INFO: Confirmed 0xd3ea66370766befa1b9896dd7760af7c31dbba258a45d7ae34837c3be66fd703 (total fees paid = 34522000000000)
INFO: Confirmed 0xbc822fb3dad0dbd950c3fb45e503885814e758d51b0d671a5fa4d72f8c1bed43 (total fees paid = 29722000000000)
INFO: Confirmed 0x56125c9b0194de87c2b6747443ffebeba1b6fd6fa732d9827ab041a5428ac685 (total fees paid = 26922000000000)
INFO: Confirmed 0x7e1e593f3628c392944d8012aace4ef2b930884de52381e5e26c57e4fae67d4e (total fees paid = 26922000000000)
INFO: Confirmed 0xfc1ebae4acaebf38ae816ebd461bd72b0296561acf8daa0212c283d174d98255 (total fees paid = 51622000000000)
INFO: Confirmed 0x27c98070f4683b7ef9e3da631c4efb98cd03804310a7c53a1cdf12bcd0526bfa (total fees paid = 34522000000000)
INFO: Confirmed 0xe14cd9423676b4be44c987d76033fa4041f33c

In [39]:
df = pd.DataFrame(result)
df

,a_balance,b_balance,simulation,subset,run,substep,timestep
0,1900000,100000,0,0,1,0,0
1,1950000,50000,0,0,1,1,1
2,1900000,100000,0,0,1,1,2
3,1950000,50000,0,0,1,1,3
4,2000000,0,0,0,1,1,4
...,...,...,...,...,...,...,...
96,1600000,400000,0,0,1,1,96
97,1550000,450000,0,0,1,1,97
98,1500000,500000,0,0,1,1,98
99,1450000,550000,0,0,1,1,99


In [40]:
px.line(df, x='timestep', y= ['a_balance', 
                              'b_balance'])

## TODO :  Demonstrate use of already deployed contracts and accounts 

In [44]:
dai = Contract('0x6B175474E89094C44Da98b954EedeAC495271d0F')

In [45]:
dai.__dict__

{'_address': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
 'contract_type': <ContractType Dai>,
 'txn_hash': None,
 '_cached_receipt': None,
 '_view_methods_': {'DOMAIN_SEPARATOR': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.DOMAIN_SEPARATOR,
  'PERMIT_TYPEHASH': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.PERMIT_TYPEHASH,
  'allowance': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.allowance,
  'balanceOf': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.balanceOf,
  'decimals': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.decimals,
  'name': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.name,
  'nonces': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.nonces,
  'symbol': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.symbol,
  'totalSupply': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.totalSupply,
  'version': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.version,
  'wards': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.wards},
 '_mutable_methods_'

In [50]:
dai.approve(dev, 100, sender = dev)

INFO: Confirmed 0x5eaeae85352631f0f4c8e201bb8ba7fb4d2f1d0e54963a94b38c9d45b839a7eb (total fees paid = 46086000000000)


<Receipt 0x5eaeae85352631f0f4c8e201bb8ba7fb4d2f1d0e54963a94b38c9d45b839a7eb>

In [52]:
dai._mutable_methods_

{'approve': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.approve,
 'burn': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.burn,
 'deny': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.deny,
 'mint': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.mint,
 'move': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.move,
 'permit': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.permit,
 'pull': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.pull,
 'push': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.push,
 'rely': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.rely,
 'transfer': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.transfer,
 'transferFrom': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.transferFrom}

In [61]:
dai.balance

0

In [62]:
dai._view_methods_

{'DOMAIN_SEPARATOR': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.DOMAIN_SEPARATOR,
 'PERMIT_TYPEHASH': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.PERMIT_TYPEHASH,
 'allowance': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.allowance,
 'balanceOf': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.balanceOf,
 'decimals': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.decimals,
 'name': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.name,
 'nonces': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.nonces,
 'symbol': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.symbol,
 'totalSupply': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.totalSupply,
 'version': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.version,
 'wards': <Dai 0x6B175474E89094C44Da98b954EedeAC495271d0F>.wards}

In [72]:
dai.balanceOf(whale)

2112188250000000000

In [70]:
whale = accounts["0xab5801a7d398351b8be11c439e05c5b3259aec9b"]

In [71]:
whale.balance

73224055451592132115

In [77]:
dai.transfer(dev, 10000, sender = whale)

<Receipt 0xb25b1193e07886361aa4fd5efb0ae92d1304b4f41542bb7e1253e53a18e4f1f8>

In [80]:
dai.balanceOf(dev)

10000